<a href="https://colab.research.google.com/github/ismaelllamur013/HR2-BotTelegramAudio/blob/main/Bot_deteccion_sonido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descripcion rapida:

Proyecto englobado en el marco del Hackathon random 2.0, una juntada entre amigos para hacer royectos interesantes. En este edicion se busco hacer un bot de Telegram con tematica libre.

En mi caso, busque realizar un bot de Telegram el cual tras ingresar un video este pueda hacer predicciones segun una categoria definida previamente, solo teniendo en cuenta el audio del video, usando como clasificador un modelo de machine learning entrenado en el momento. A su vez se aprobecho para evaluar que otras funciones se pueden utilizar en los bot de telegram, siendo esta la primera experiencia programando uno.

# Instalar librerias

In [ ]:
pip install pydub

In [ ]:
pip install librosa

In [ ]:
pip install python-telegram-bot==13.7

In [ ]:
pip install moviepy

# Importar librerias

In [ ]:
from pydub import AudioSegment
import librosa
import numpy as np
from scipy.io import wavfile


from telegram import Update
from telegram.ext import Updater, CommandHandler,  MessageHandler, CallbackQueryHandler, CallbackContext, Filters
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
import os
import time
from moviepy.editor import VideoFileClip

# Funciones


"remuentrar_audio" Es una funcion que utilizaremos para definir la frecuencia de muestreo del audio, obtenido del video, que se ingresara a nuestro modelo de Machine Learning. Se ingresa input_audio (nombre del archivo inicial), output_audio (nombre del archivo final), nueva_frecuencia (frecuencia de muestro)

In [ ]:
def remuestrear_audio(input_audio, output_audio, nueva_frecuencia):
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(input_audio)
    # Realizar el re-muestreo a la nueva frecuencia
    audio_remuestreado = audio.set_frame_rate(nueva_frecuencia)
    # Guardar el audio re-muestreado en un nuevo archivo
    audio_remuestreado.export(output_audio, format="wav")

"ajustar_longitud_audio" Al ingresar a nuestro modelo un audio necesitaremos que tenga una longuitud especifica, est la ingresaremos en longitud_deseada que sera la frecuencia de muestreo por los segundos que aceptemos

In [ ]:
def ajustar_longitud_audio(audio_path, longitud_deseada):
    # Cargar el archivo de audio
    audio, sample_rate = librosa.load(audio_path, sr=None)

    # Ajustar la longitud del audio
    if len(audio) > longitud_deseada:
        audio_ajustado = audio[:longitud_deseada]
    elif len(audio) < longitud_deseada:
        padding = longitud_deseada - len(audio)
        audio_ajustado = np.pad(audio, (0, padding), mode='constant')
    else:
        audio_ajustado = audio
    return audio_ajustado, sample_rate

"extraer_caracteristicas" La utilizaremos para elegir la caracteristica que evaluaremos en nuestro modelo, en este caso en la señal de audio completa

In [ ]:
def extraer_caracteristicas(audio_path):
    # Implementa aquí la extracción de características del audio
    # Puedes utilizar bibliotecas como librosa para extraer características
    # Aquí, se utiliza un ejemplo simple, considerando la longitud del audio
    _, audio_data = wavfile.read(audio_path)
    #longitud_audio = len(audio_data)
    #return np.array([longitud_audio])
    return audio_data

"start" Es la funcion que inicializado el bot o utilizando el comando /start, nos dara un cartel de inicio y nos permitir elegir entre 4 botones distintos con sus respectivas respuests cada uno

In [ ]:
def start(update: Update, context: CallbackContext) -> None:
    user = update.effective_user
    update.message.reply_text(f'Hola {user.first_name}! Que pasa perro, estas para dejar la carrera?')

    # Crear un teclado de respuesta en línea con un botón
    keyboard = [[InlineKeyboardButton("Saludar", callback_data='saludar')],
                [InlineKeyboardButton("Llorar", callback_data='llorar')],
                [InlineKeyboardButton("Invitar un matesito", callback_data='matesito')],
                [InlineKeyboardButton("Secreto", callback_data='secreto')]]

    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text('¿Qué te gustaría hacer?', reply_markup=reply_markup)

"button_click" define que sucedera elegido alguno de los botones previamente nombrados

In [ ]:
# Define la función para manejar la pulsación de botones en el teclado de respuesta en línea
def button_click(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()

    if query.data == 'saludar':
        user = update.effective_user
        query.edit_message_text(f'Saludos, {user.first_name}!')

    if query.data == 'llorar':
        user = update.effective_user
        query.edit_message_text(f'Eeehh perro que te pasa')

    if query.data == 'matesito':
        user = update.effective_user
        query.edit_message_text(f'🧉')

    if query.data == 'secreto':
        user = update.effective_user
        query.edit_message_text(f'mmmmmmmmmmmm')

In [ ]:
# Define la función para el comando /help
def help_command(update, context):
    update.message.reply_text('Puedes enviar mensajes y este bot responderá.')

"handle_video" la funcion recibe el video y lo guarda en una carpeta definida previamente, y avisa que se recibio con exito

In [ ]:
# Directorio donde se guardarán los videos
VIDEO_DIRECTORY = "videos"
# Crear el directorio si no existe
os.makedirs(VIDEO_DIRECTORY, exist_ok=True)
# Define la función para manejar mensajes de video
def handle_video(update: Update, context: CallbackContext) -> None:
    video_file = update.message.video.file_id
    video_caption = update.message.caption
    # Proporciona un nombre específico para el video (puedes personalizar esto)
    video_filename = "video_personalizado.mp4"
    # Descarga el video
    video = context.bot.get_file(video_file, timeout=20)
    video_path = os.path.join(VIDEO_DIRECTORY, f"{video_file}.mp4")
    video.download(video_path)
    # Responde al usuario
    update.message.reply_text(f"¡Video guardado con éxito!")

"send_audio" recibe el video, obtiene el audio, lo guarda y envia al bot para que la persona pueda descargarlo

In [ ]:
def extraer_audio(input_video, output_audio):
    # Cargar el archivo de video
    video_clip = VideoFileClip(input_video)
    # Extraer el audio del video
    audio_clip = video_clip.audio
    # Guardar el audio en un archivo separado
    audio_clip.write_audiofile(output_audio)
    # Cerrar los clips para liberar recursos
    audio_clip.close()
    video_clip.close()

In [ ]:
# Define la función para el comando /audio
def send_audio(update: Update, context: CallbackContext) -> None:
    # Ejemplo de uso
    input_video = "/content/videos/"+os.listdir("videos")[0]
    output_audio = "audio_personalizado.wav"
    extraer_audio(input_video, output_audio)
    update.message.reply_text(update.message.text)
    # Obtén el identificador del chat
    chat_id = update.message.chat_id
    # Construye la ruta del archivo de audio
    audio_file_path = os.path.join("/content", "audio_personalizado.wav")
    # Envía el archivo de audio al chat
    context.bot.send_audio(chat_id=chat_id, audio=open(audio_file_path, 'rb'), title="Audio de ejemplo")

"predictor" es la funcion en donde estara nuesto modelo previamente entrenado, se le ingresara el audio del video y se lo evaluara con el fin de descubir si pertenece o la categoria del modelo entrenado

In [ ]:
def predictor(update: Update, context: CallbackContext) -> None:
    # Ejemplo de uso
    input_audio = "audio_personalizado.wav"
    output_audio_remuestreado = "audio_remuestreado.wav"
    nueva_frecuencia = 44100  # Puedes ajustar la frecuencia según tus necesidades
    remuestrear_audio(input_audio, output_audio_remuestreado, nueva_frecuencia)
    # Longitud deseada para todos los archivos de audio (ajusta según tus necesidades)
    longitud_deseada = 44100*30 # Ejemplo: 1 segundo a 44.1 kHz
    # Ruta del archivo de entrada y salida (ajusta según tus necesidades)
    audio_input_path = "audio_remuestreado.wav"
    audio_output_path = "audio_fin.wav"
    # Cargar y ajustar la longitud del audio
    audio_ajustado, sample_rate = ajustar_longitud_audio(audio_input_path, longitud_deseada)
    # Exportar el audio ajustado a un nuevo archivo WAV
    wavfile.write(audio_output_path, sample_rate, audio_ajustado)
    # ... (Código anterior para obtener coeficientes e intercepto)
    audio_paths_tipo = ["audio_fin.wav"]
    X = np.array([extraer_caracteristicas(path) for path in audio_paths_tipo])
    # Nuevos datos de entrada para hacer predicciones
    nuevos_datos = X[-1]
    # Calcular el valor de decisión sin utilizar el objeto modelo
    decision_values = np.dot(coeficientes.reshape(1,-1), nuevos_datos.T) - intercepto.reshape(1,-1)
    # Aplicar la función de signo para obtener las predicciones (1 para clase positiva, -1 para clase negativa)
    predicciones = np.sign(decision_values)
    # Imprimir las predicciones
    if predicciones == 1:
        update.message.reply_text(f"Una sodita")
    else:
        update.message.reply_text(f"ESO NO ES UNA Sodita, PAPU")

    print("Predicciones:", predicciones)


Se requiere introducir los coeficientes del modelo (en este caso una maquina de soportes) y el token de nuestro bot

In [ ]:
archivo_txt = 'coeficientes.txt'
# Carga los datos desde el archivo de texto usando numpy.loadtxt
coeficientes = np.loadtxt(archivo_txt)

archivo_txt = 'intercepto.txt'
# Carga los datos desde el archivo de texto usando numpy.loadtxt
intercepto = np.loadtxt(archivo_txt)

In [ ]:
# Configura el token de tu bot aquí
TOKEN = 'AQUI SE INTRODUCE EL TOKEN'

"main" es nuestra funcion principal

In [ ]:
def main():
    # Crea el objeto Updater y pasa el token de tu bot
    updater = Updater(TOKEN, use_context=True)

    # Obtén el despachador para registrar manejadores
    dp = updater.dispatcher

    # Registra los manejadores de comandos
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("ayuda", help_command))
    dp.add_handler(MessageHandler(Filters.video, handle_video))
    dp.add_handler(CommandHandler("audio", send_audio))
    dp.add_handler(CommandHandler("sodita", predictor))


    # Registra el manejador para manejar la pulsación de botones en el teclado de respuesta en línea
    dp.add_handler(CallbackQueryHandler(button_click))

    # Inicia el bot
    updater.start_polling()

    # Ejecuta el bot hasta que se presione Ctrl+C
    updater.idle()

if __name__ == '__main__':
    main()

MoviePy - Writing audio in audio_personalizado.wav


MoviePy - Done.


Predicciones: [[1.]]
